# Advanced Programming Summative Assignment

Please see the README file (README.md) for a full overview of the project, installation instructions and running instructions. 

## Contents

#### 1.0 [Import Statements](#1.0-Import-statements)

#### 2.0 [Some test....](#2.0-Some-test)

2.1 [Some other test](#2.1-Some-other-test)

### 1.0 Import statements

In [4]:
# import statements
import pandas as pd
import numpy as np
import json

In [69]:
# read CSV files
df_airports = pd.read_csv ('data/airports.csv', nrows=10)
df_runways = pd.read_csv ('data/runways.csv', index_col=['id'], nrows=10)
df_frequencies = pd.read_csv ('data/airport-frequencies.csv', index_col=['id'], nrows=10)

#remove any rows with all data missing
df_airports.dropna(how='all')
#remove any duplicated rows
df_airports.drop_duplicates()
# drop largely empty or irrelevant columns
df_airports = df_airports.drop(['keywords','home_link'], axis='columns')
# filter out closed airports - may need to force a copy, not sure yet
df_airports = df_airports[(df_airports.type != 'closed')]
# add columns for small, medium and large with binary values
types = ['small_airport','medium_airport','large_airport']
zero_matrix = np.zeros((len(df_airports), len(types)))





df_airports = df_airports.join(pd.get_dummies(col_values))

print(df_airports)

       id ident           type                             name  latitude_deg  \
0    6523   00A       heliport                Total Rf Heliport     40.070801   
1  323361  00AA  small_airport             Aero B Ranch Airport     38.704022   
2    6524  00AK  small_airport                     Lowell Field     59.947733   
3    6525  00AL  small_airport                     Epps Airpark     34.864799   
5  322127  00AS  small_airport                   Fulton Airport     34.942803   
6    6527  00AZ  small_airport                   Cordes Airport     34.305599   
7    6528  00CA  small_airport          Goldstone (GTS) Airport     35.354740   
8  324424  00CL  small_airport              Williams Ag Airport     39.427188   
9  322658  00CN       heliport  Kitchen Creek Helibase Heliport     32.727374   

   longitude_deg  elevation_ft  continent iso_country iso_region  \
0     -74.933601            11        NaN          US      US-PA   
1    -101.473911          3435        NaN          US

In [63]:
# write to JSON
airports_json = df_airports.to_json(orient = 'records')
print(airports_json)

[{"id":6523,"ident":"00A","type":"heliport","name":"Total Rf Heliport","latitude_deg":40.0708007812,"longitude_deg":-74.9336013794,"elevation_ft":11,"continent":null,"iso_country":"US","iso_region":"US-PA","municipality":"Bensalem","scheduled_service":"no","gps_code":"00A","iata_code":null,"local_code":"00A","wikipedia_link":null},{"id":323361,"ident":"00AA","type":"small_airport","name":"Aero B Ranch Airport","latitude_deg":38.704022,"longitude_deg":-101.473911,"elevation_ft":3435,"continent":null,"iso_country":"US","iso_region":"US-KS","municipality":"Leoti","scheduled_service":"no","gps_code":"00AA","iata_code":null,"local_code":"00AA","wikipedia_link":null},{"id":6524,"ident":"00AK","type":"small_airport","name":"Lowell Field","latitude_deg":59.947733,"longitude_deg":-151.692524,"elevation_ft":450,"continent":null,"iso_country":"US","iso_region":"US-AK","municipality":"Anchor Point","scheduled_service":"no","gps_code":"00AK","iata_code":null,"local_code":"00AK","wikipedia_link":nul

### 2.0 Some test

#### 2.1 Some other test